In [205]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.cluster import AgglomerativeClustering, HDBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
import umap

%matplotlib qt5
sns.set_style('whitegrid')
sns.set_context('paper')
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 0.5

custom_12 = [
    "#1f77b4",  # blue
    "#ff7f0e",  # orange
    "#2ca02c",  # green
    "#d62728",  # red
    "#9467bd",  # purple
    "#8c564b",  # brown
    "#e377c2",  # pink
    "#000000",  # black
    "#bcbd22",  # olive
    "#17becf",  # cyan
    "#aec7e8",  # light blue
    "#ffbb78",  # light orange
]

In [206]:
from typing import Optional
import numbers

def auto_opt_pd_dtypes(df_: pd.DataFrame, inplace=False) -> Optional[pd.DataFrame]:
    """ Automatically downcast Number dtypes for minimal possible,
        will not touch other (datetime, str, object, etc)
        :param df_: dataframe
        :param inplace: if False, will return a copy of input dataset
        :return: `None` if `inplace=True` or dataframe if `inplace=False`
    """
    df_temp = df_ if inplace else df_.copy()
    print(df_temp.info())

    for col in df_temp.columns:
        # integers
        if issubclass(df_temp[col].dtypes.type, numbers.Integral):
            # unsigned integers
            if df_temp[col].min() >= 0:
                df_temp[col] = pd.to_numeric(df_temp[col], downcast='unsigned')
            # signed integers
            else:
                df_temp[col] = pd.to_numeric(df_temp[col], downcast='integer')
        # other real numbers
        elif issubclass(df_temp[col].dtypes.type, numbers.Real):
            df_temp[col] = pd.to_numeric(df_temp[col], downcast='float')

        elif issubclass(df_temp[col].dtypes.type, np.object_):
            df_temp[col] = pd.Categorical(df_temp[col])

    print(df_temp.info())
    if not inplace:
        return df_temp

In [207]:
df = pd.read_csv('kc_house_data.csv', header='infer', delimiter=',', parse_dates=['date'])
df = auto_opt_pd_dtypes(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  int64         
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  int64         
 6   sqft_lot       21613 non-null  int64         
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  int64         
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  int64         
 13  sqft_basement  21613 non-null  int64         
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   2161

In [208]:
df.describe()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,21613,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,...,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,2014-10-29 04:38:01.959931648,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,...,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560051,-122.213898,1986.552492,12768.455652
min,1.000102e+06,2014-05-02 00:00:00,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,...,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155899,-122.518997,399.000000,651.000000
25%,2.123049e+09,2014-07-22 00:00:00,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,...,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471001,-122.328003,1490.000000,5100.000000
50%,3.904930e+09,2014-10-16 00:00:00,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,...,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230003,1840.000000,7620.000000
75%,7.308900e+09,2015-02-17 00:00:00,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,...,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678001,-122.125000,2360.000000,10083.000000
max,9.900000e+09,2015-05-27 00:00:00,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,...,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777599,-121.315002,6210.000000,871200.000000
std,2.876566e+09,NaN,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,...,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631


In [209]:
df['sqft_basement'] = df['sqft_basement'].replace(0, np.nan)
df['yr_renovated'] = df['yr_renovated'].replace(0, np.nan)
df['yr_renovated'] = df['yr_renovated'].replace(1, np.nan)

In [210]:
cols = [col for col in df.columns if col not in ('id', 'date')]
cols_log = ['price', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']

# view is mostly 0
cols_pairplot = [col for col in cols if col not in ('sqft_basement', 'zipcode', 'lat', 'long', 'view', 'grade', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'floors', 'waterfront', 'condition')]

In [211]:
df = df.loc[df['bedrooms'] < 30]

In [212]:
df_log = df.copy()
df_log[cols_log] = np.log10(df_log[cols_log])
df_log['yr_renovated_new'] = 2016 - df_log['yr_renovated']
df_log['yr_built_new'] = 2016 - df_log['yr_built']

In [213]:
df_modelling = df_log.copy()

df_modelling = df_modelling[[col for col in df_modelling.columns if col not in (
'zipcode', 'yr_built', 'yr_renovated', 'bathrooms', 'sqft_living', 'grade', 'sqft_living15', 'sqft_lot15')]]

df_modelling = df_modelling.sort_values('date').groupby('id').agg('last').reset_index()
df_modelling.loc[df_modelling['yr_renovated_new'].isna(), 'yr_renovated_new'] = df_modelling['yr_built_new']
df_modelling['sqft_basement'] = df_modelling['sqft_basement'].fillna(0)

In [214]:
from sklearn.preprocessing import MinMaxScaler

cols_norm = [col for col in df_modelling.columns if col not in ('id', 'date')]
scaler = MinMaxScaler()
df_normalized = df_modelling.copy()
df_normalized[cols_norm] = scaler.fit_transform(df_modelling[cols_norm])

In [215]:
# 
# num_data = df_modelling[cols_norm]
# 
# minmaxscaler = MinMaxScaler()
# standardscaler = StandardScaler()
# 
# x = pd.DataFrame(data=standardscaler.fit_transform(num_data[cols_norm]), columns=cols_norm)
# 
# pca_input = x.dropna()
# model = PCA(n_components=5)
# pca_data = model.fit_transform(pca_input)
# 
# num_data[['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5']] = pca_data
# 
# sns.scatterplot(data=num_data.sort_values(by='price', ascending=True), x='PCA 1', y='PCA 2', hue='price', palette='RdYlGn')
# plt.xlabel('PCA component 1')
# plt.ylabel('PCA component 2')
# plt.title('PCA')
# plt.tight_layout()
# plt.show()
# 
# print(f'Explained variance by first 5 compononents: {sum(model.explained_variance_ratio_):.3f}')

In [37]:
from sklearn.cluster import HDBSCAN

reducer = umap.UMAP(n_components=6)
data = df_modelling[[col for col in df_modelling.columns if col not in ('id', 'date')]].values
scaled_data = StandardScaler().fit_transform(data)
embedding = reducer.fit_transform(scaled_data)
X = embedding
mcs = round(len(df_modelling) * 0.025)
min_samples = round(mcs * 0.25)
clusterer = HDBSCAN(
    min_cluster_size=mcs,
    min_samples=min_samples,
    metric="euclidean"
)
labels = clusterer.fit_predict(X)

## 2D UMAP Clusters

In [38]:
data = df_modelling[cols_norm].values
scaled_data = StandardScaler().fit_transform(data)
reducer = umap.UMAP()
embedding = reducer.fit_transform(scaled_data)
df_results = pd.DataFrame(data=embedding)
df_results = pd.concat([df_results, df_modelling], axis=1)
df_results['label'] = labels

In [40]:
col = 'label'
results_sorted = df_results.sort_values(col)
results_sorted = results_sorted.loc[results_sorted['label'] != -1]
ax = sns.scatterplot(x=results_sorted.iloc[:, 0], y=results_sorted.iloc[:, 1], s=10, linewidth=0, hue=results_sorted[col], palette='tab10', alpha=0.2)

legend = ax.get_legend()
for handle in legend.legend_handles:
    handle.set_alpha(1)
    
# plt.title('Hierarchical Clustering on projected Data (n=6, linkage=ward)')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.tight_layout()
# plt.savefig('hierarchical_n6_ward_projected')

In [32]:
x = results_sorted['lat']
y = results_sorted['long']

ax = sns.scatterplot(x=x, y=y, hue=results_sorted['label'].astype(str), s=5, alpha=0.5, palette='tab10_r', linewidth=0)

legend = ax.get_legend()
for handle in legend.legend_handles:
    handle.set_alpha(1)
    
plt.title('Hierarchical Clustering on projected Data (n=6, linkage=ward)')
plt.xlabel('Lat.')
plt.ylabel('Long.')
plt.tight_layout()
# plt.savefig('hierarchical_n6_ward_coords')

In [19]:
df_settings = pd.read_csv('HDBSCAN_parameter_results.csv')

In [45]:
# cols_log.remove('sqft_living')
# cols_log.remove('sqft_living15')
# cols_log.remove('sqft_lot15')
# df_results[cols_log] = 10**df_results[cols_log]
df_results['renovated'] = df_results['yr_built_new'] != df_results['yr_renovated_new']
cluster_data = df_results.groupby('label').agg(['mean'])
cluster_data

,0,1,id,date,price,bedrooms,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,lat,long,yr_renovated_new,yr_built_new,renovated
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
label,,,,,,,,,,,,,,,,,,
-1,-2.056331,3.706224,5.522237e+09,2014-10-31 22:11:40.884955648,1.356166e+06,3.000000,19733.924779,1.480088,0.721239,2.716814,3.398230,2249.229980,514.517699,47.582760,-122.212082,43.725664,51.672566,0.185841
0,11.473857,-6.403739,4.429368e+09,2014-10-27 23:30:43.253234688,5.064622e+05,2.837338,1604.131238,2.990758,0.000000,0.007394,3.000000,1547.105347,28.295749,47.665104,-122.339432,18.009242,18.009242,0.000000
1,8.305052,11.449243,4.391379e+09,2014-10-25 23:45:25.362517248,4.394412e+05,3.151573,16175.130506,1.315321,0.000000,0.010944,4.187962,1630.372925,1.218057,47.531063,-122.212463,65.915185,66.749384,0.019973
2,16.338198,5.948574,4.610391e+09,2014-11-01 02:24:56.605122048,4.847781e+05,3.297915,15217.201072,1.663311,0.000000,0.005837,2.983800,2031.234741,6.483502,47.544739,-122.171043,41.569744,43.652412,0.038952
3,0.749989,7.643003,4.533159e+09,2014-10-21 08:13:39.430950912,5.612198e+05,3.593685,12860.260583,1.146773,0.000000,0.031228,4.269951,1401.443726,779.975017,47.577610,-122.256195,71.692575,72.327550,0.012838
4,2.525029,-0.548284,4.613020e+09,2014-11-03 19:31:48.370044160,5.545016e+05,3.549437,12967.360744,1.350220,0.000000,0.036711,3.006853,1558.687256,674.802496,47.588100,-122.245598,48.889623,53.641214,0.071953
5,7.865779,2.358121,4.774841e+09,2014-10-28 08:28:48.000000000,8.913216e+05,3.613333,25574.193333,1.530909,0.000000,2.480606,3.491515,2197.389160,648.890303,47.565762,-122.239136,53.442424,58.107879,0.083030


In [216]:
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

# Hierarchical clustering

In [241]:
reducer = umap.UMAP(n_components=12)
data = df_modelling[[col for col in df_modelling.columns if col not in ('id', 'date')]].values
scaled_data = StandardScaler().fit_transform(data)
embedding = reducer.fit_transform(scaled_data)
X = embedding

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=None, n_clusters=16, linkage='average')

model = model.fit(X)
labels = model.labels_

In [255]:
data = df_modelling[cols_norm].values
scaled_data = StandardScaler().fit_transform(data)
reducer = umap.UMAP()
embedding = reducer.fit_transform(scaled_data)
df_results = pd.DataFrame(data=embedding)
df_results = pd.concat([df_results, df_modelling], axis=1)
df_results['label'] = labels
# df_results['label'] = df_results['label'].astype(str)

In [256]:
# cols_log_post = list(set(cols_log) & set(df_modelling.columns))
# df_results[cols_log_post] = 10**df_results[cols_log_post]
# df_results[cols_log_post] = df_results[cols_log_post].replace(1, 0)

df_results['renovated'] = df_results['yr_built_new'] != df_results['yr_renovated_new']
df_results['years_until_renovation'] = df_results['yr_built_new'] - df_results['yr_renovated_new']
df_results['years_until_renovation_none'] = df_results['years_until_renovation'].replace(0, None)
cluster_data = df_results.groupby('label').agg(['mean'])
cluster_sizes = df_results.groupby('label')['id'].agg(['count'])
cluster_data['cluster_size'] = cluster_sizes
cluster_data.reset_index(inplace=True)
cluster_data.fillna(0, inplace=True)

C:\Users\bo112\AppData\Local\Temp\ipykernel_70640\3065897734.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cluster_data.fillna(0, inplace=True)


In [257]:
cluster_data.to_csv('results_hierarchical_ncomp8_nclusters12_linkage_complete_log_fixed_age.csv', index=False, sep=';', decimal=',')

In [252]:
plot_type = 'kde'
# plot_type = 'box'

if plot_type == 'box':
    fig, axs = plt.subplots(ncols=2, nrows=5, figsize=(18, 10), sharex=True)
elif plot_type == 'kde':
    fig, axs = plt.subplots(ncols=2, nrows=5, figsize=(18, 10))
else:
    pass

index = 0
axs = axs.flatten()

cols_remove = ['bedrooms', 'floors', 'condition', 'view', 'renovated', 'waterfront', 'sqft_basement']
cols_kde = list(set(df_results.iloc[:, 4:].columns) - set(cols_remove))

for k,v in df_results[cols_kde].items():
    try:
        if k in ['bedrooms', 'floors', 'condition', 'view', 'renovated']:
            bw = 5
        else:
            bw = 1
        scale = True if k in cols_log else False
        
        if k == 'label':
            axs[-1].bar(
                range(16),
                df_results.groupby('label')['id'].agg(['count'])['count'].values,
                color=sns.color_palette('tab20')[:16],
                edgecolor="none"
            )
            axs[-1].set_xticks(range(16))
            axs[-1].set_xticklabels(range(16))
            axs[-1].set_xlabel('Cluster ID')
            axs[-1].set_ylabel('Cluster size')
            index -= 1
        else:
            if plot_type == 'kde':
                sns.kdeplot(x=k, data=df_results, ax=axs[index], log_scale=scale, hue='label', common_norm=False, palette='tab20', multiple='fill', legend=False, linewidth=0, fill=True, bw_adjust=bw)
                if index not in (0, 5):
                    axs[index].set_ylabel(None)
            elif plot_type == 'box':
                sns.boxplot(df_results, x='label', y=k, ax=axs[index], hue='label', palette='tab20', log_scale=scale, legend=False)
                axs[index].set_xlabel('Cluster ID')
            else:
                pass
    except ValueError as e:
        pass
    index += 1
plt.tight_layout()
plt.show()

C:\Users\bo112\AppData\Local\Temp\ipykernel_70640\3312472137.py:39: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(x=k, data=df_results, ax=axs[index], log_scale=scale, hue='label', common_norm=False, palette='tab20', multiple='fill', legend=False, linewidth=0, fill=True, bw_adjust=bw)
C:\Users\bo112\AppData\Local\Temp\ipykernel_70640\3312472137.py:39: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(x=k, data=df_results, ax=axs[index], log_scale=scale, hue='label', common_norm=False, palette='tab20', multiple='fill', legend=False, linewidth=0, fill=True, bw_adjust=bw)


In [56]:
plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=5)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [246]:
results_sorted = df_results.sort_values('label')
results_sorted['label'] = results_sorted['label'].astype(str)
ax = sns.scatterplot(x=results_sorted.iloc[:, 0], y=results_sorted.iloc[:, 1], s=10, linewidth=0, hue=results_sorted['label'],
                     palette='tab20', alpha=0.3)

ax.legend(ncol=4, title='Cluster ID', loc='upper left')
legend = ax.get_legend()
for handle in legend.legend_handles:
    handle.set_alpha(1)

plt.title('Hierarchical Clustering on projected Data')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.tight_layout()
# plt.savefig('hierarchical_n6_ward_projected')

In [248]:
ax = sns.scatterplot(results_sorted, x='lat', y='long', hue='label', s=5, alpha=0.7, palette='tab20', linewidth=0)

ax.legend(ncol=4, title='Cluster ID', loc='upper left')
legend = ax.get_legend()
for handle in legend.legend_handles:
    handle.set_alpha(1)

plt.title('Hierarchical Clustering on Coordinate Data')
plt.xlabel('Lat.')
plt.ylabel('Long.')
plt.tight_layout()
# plt.savefig('hierarchical_n6_ward_coords')


In [235]:
from tqdm import tqdm

linkage_methods = ['average', 'complete', 'ward']
df_settings = pd.DataFrame(columns=['linkage', 'n_components', 'n_clusters', 'silhouette_score', 'dbi_score'])

for n in tqdm((6, 8, 10, 12)):
    reducer = umap.UMAP(n_components=n)
    data = df_modelling[[col for col in df_modelling.columns if col not in ('id', 'date')]].values
    scaled_data = StandardScaler().fit_transform(data)
    embedding = reducer.fit_transform(scaled_data)
    X = embedding
    for linkage in linkage_methods:
        print(linkage)
        for n_clusters in range(6, 21, 2):
            model = AgglomerativeClustering(distance_threshold=None, n_clusters=n_clusters, linkage=linkage)

            model = model.fit(X)
            labels = model.labels_

            temp = pd.DataFrame(data={'linkage': linkage,
                                      'n_components': n,
                                      'n_clusters': n_clusters,
                                      'silhouette_score': silhouette_score(X, labels),
                                      'dbi_score': davies_bouldin_score(X, labels)
                                      },
                                index=[0])
            df_settings = pd.concat([df_settings, temp]).reset_index(drop=True)

  0%|          | 0/4 [00:00<?, ?it/s]

average


C:\Users\bo112\AppData\Local\Temp\ipykernel_70640\47914910.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_settings = pd.concat([df_settings, temp]).reset_index(drop=True)


complete
ward


 25%|██▌       | 1/4 [07:22<22:06, 442.31s/it]

...

 50%|█████     | 2/4 [14:51<14:52, 446.09s/it]

average
complete
ward


 75%|███████▌  | 3/4 [33:36<12:36, 756.20s/it]

average
complete
ward


100%|██████████| 4/4 [57:29<00:00, 862.44s/it] 


In [236]:
df_settings.to_csv('param_search_hierarchical_fixed_age.csv', index=False, sep=';', decimal=',')

In [240]:
df_settings = pd.read_csv('param_search_hierarchical_fixed_age.csv', sep=';', decimal=',')
fig, axes = plt.subplots(2, 3, sharex=True, figsize=(15, 8))

for col, linkage in enumerate(('average', 'complete', 'ward')):
    temp = df_settings.loc[df_settings['linkage'] == linkage]
    for row, score in enumerate(('silhouette_score', 'dbi_score')):
        sns.lineplot(temp, x='n_clusters', y=score, hue='n_components', palette='coolwarm', ax=axes[row, col])
        if row == 0:
            axes[row, col].set_title(f'linkage: {linkage}')
        if score == 'silhouette_score':
            axes[row, col].set_ylim(0.45, 0.7)
        else:
            axes[row, col].set_ylim(0.4, 1)
        plt.tight_layout()
plt.show(block=True)